In [1]:
from VFXTesting.lol_champ_reader import LoLChampReader
import dotenv
import pathlib
import random
import google.generativeai as genai


media = pathlib.Path("Data/Videos")

LOL_CHAMP_READER = LoLChampReader()
VIDEO_DIR = "Data/Videos"
SCHEMA_PATH = "Schemas/schema.json"

gemini_key = dotenv.get_key('.env', 'GEMINI_KEY')
if not gemini_key:
    raise ValueError("GEMINI_KEY environment variable not set.")
genai.configure(api_key=gemini_key)
MODEL = "gemini-1.5-pro-002"
model = genai.GenerativeModel(MODEL)

In [2]:
import json

def load_schema():
    with open(SCHEMA_PATH, 'r') as f:
        schema = json.load(f)
    return schema

In [6]:
import os

def get_random_videos(video_directory, sample_size=10):
    all_videos = [f for f in os.listdir(video_directory) if f.endswith('.webm') and parse_video_filename(f) is not None]
    sample_videos = random.sample(all_videos, min(sample_size, len(all_videos)))
    return sample_videos


def parse_video_filename(filename):
    """
    Helper function to extract champion name and ability from the file name
    """
    base_name = os.path.splitext(filename)[0]
    try:
        split = base_name.split('_')
        champ_name = split[0]
        ability = split[1]
        if ability not in {'Q', 'W', 'E', 'R'}:
            return None
        return champ_name, ability
    except ValueError:
        return None

In [8]:
import time
from typing import Literal


def generate_ground_truth(schema, video_file, champ_name, ability: Literal['Q', 'W', 'E', 'R'] = 'Q'):
    """
    Generates ground truth data for a video using Gemini.
    """
    ability_info = LOL_CHAMP_READER.get_champ_ability(champ_name, ability)
    if not ability_info:
        return None

    ability_info_str = json.dumps(ability_info, indent=2)

    video = genai.upload_file(media / video_file)
    while video.state.name == "PROCESSING":
        print("processing video...")
        time.sleep(10)
        video = genai.get_file(video.name)

    prompt = [
        video,
        f"Given the contents of this video and the following JSON ability info: {ability_info_str}",
        f"Reply only with the key vfx_properties information using the JSON schema {schema}",
    ]

    response = model.generate_content(prompt)
    return response.text

In [9]:
schema = load_schema()
videos = get_random_videos(VIDEO_DIR, sample_size=1)

for video in videos:
    try:
        champ_name, ability = parse_video_filename(video)
        ground_truth = generate_ground_truth(schema, video, champ_name, ability)
        print(video, ground_truth)
    except ValueError as e:
        print(f"Skipping file {video}: {e}")

processing video...
Amumu_E_.webm ```json
{
  "vfx_properties": {
    "startTime": 0.0,
    "endTime": 0.5,
    "shape": "Circular Area of Effect",
    "color": {
      "Red": "#8B0000",
      "Orange": "#FFA500"
    },
    "projectileCount": 0
  }
}

```
